# Hierarchical Risk Parity - A HBDC Research Note

## Introduction

Risk-Parity (RP) allocates proportional to inverse variance: $w_k = \frac{\nu_k^{-1}}{\sum_k\nu_k^{-1}}$. This completely ignores correlation information.  

Mean-Variance-Optimization takes into account full covariance matrix. $w_k = \Sigma^{-1}\mu$. However, MVO weights are unstable (big changes for small changes in covariances) and frequently extreme (corner solutions).

Hierarchical Risk-Parity sits in between these two extremes. It takes into account dependency information in somewhat "coarse" way by grouping assets that are similar in the sense of a distance metric.

In [1]:
using Distributions, Random, Statistics, LinearAlgebra
Random.seed!(1234)

TaskLocalRNG()

## An Example

HRP is best understood through a numerical example. Consider three assets with the following covariance matrix of returns.

In [184]:
Σ = [0.0225 0.00900343 0.00946224; 0.00900343 0.04 0.0137452; 0.00946224 0.0137452 0.0225]

3×3 Matrix{Float64}:
 0.0225      0.00900343  0.00946224
 0.00900343  0.04        0.0137452
 0.00946224  0.0137452   0.0225

We need a distance metric since neither covariance nor correlation is a proper distance metric. It can be shown that $d(i,j) = \sqrt{(1-\rho_{ij})/2}$ is a proper distance metric. We convert the covariance matrix into a distance matrix by first extracting the correlation matrix and then converting each correlation entry into a distance measure with the formula above.

In [185]:
function cov2corr(C)
    σinv = diagm(1.0 ./ sqrt.(diag(C)))
    return σinv * C * σinv
end

cov2corr (generic function with 1 method)

In [186]:
R = cov2corr(Σ)

3×3 Matrix{Float64}:
 1.0       0.300114  0.420544
 0.300114  1.0       0.458173
 0.420544  0.458173  1.0

In [202]:
D = sqrt.(0.5*(1.0.-R))

3×3 Matrix{Float64}:
 0.0       0.59156   0.538264
 0.59156   0.0       0.520493
 0.538264  0.520493  0.0

Under risk-parity the allocations to three assets would only depend on the diagonal entries in $\Sigma$.

In [188]:
function ivp(C::Matrix{Float64})
    iv = 1.0 ./ diag(C)
    return iv ./ sum(iv)
end

ivp (generic function with 3 methods)

In [189]:
wrp = ivp(Σ)

3-element Vector{Float64}:
 0.3902439024390244
 0.21951219512195122
 0.3902439024390244

For this portfolio, under hierarchical risk-parity, we first find the pair that is closest in distance and treat it as one asset. From the distance matrix D, we see that assets 2 and 3 are the closest. We group them together into a portfolio (say, asset 4). The mix of 2 and 3 in this portfolio is done as per risk-parity.

In [190]:
w2, w3 = ivp(Σ[2:3, 2:3])

2-element Vector{Float64}:
 0.36
 0.64

Variance of this asset 4 can be found from the covariance matrix.

In [191]:
function clustervar(C, cluster)
    V = C[cluster, cluster]
    u = ivp(V)
    return dot(u' * V, u)
end

clustervar (generic function with 1 method)

In [193]:
v4 = clustervar(Σ, [2,3])

0.02073378816

Now we are left with two assets 1 and 4. We conclude the allocation process by simply allocating based on risk parity between assets 1 and 4.

In [194]:
w1, w4 = ivp([Σ[1,1], v4])

2-element Vector{Float64}:
 0.47957370941607536
 0.5204262905839246

But, we know that asset 4 is just a combination of 36% of asset 2 and 64% of asset 3. So we cascade the allocation to 4 down to 2 and 3.

In [195]:
w2, w3 = w4 * [w2, w3]

2-element Vector{Float64}:
 0.18735346461021288
 0.3330728259737118

This gives the final HRP allocation of

In [196]:
whrp = [w1, w2, w3]

3-element Vector{Float64}:
 0.47957370941607536
 0.18735346461021288
 0.3330728259737118

Compare this with RP. Asset 1 is dissimilar to both assets 2 and 3, so it receives a relatively higher allocation in HRP.

In [197]:
[wrp whrp]

3×2 Matrix{Float64}:
 0.390244  0.479574
 0.219512  0.187353
 0.390244  0.333073

## Summary of HRP methodology

1. Compute agglomorative clustering of assets in the given portfolio. That is, at every stage cluster or group pair of assets or clusters from the previous stage that are closest in the distance metric. This gives a binary tree. For example, in a portfolio of 4 assets numbered 1 to 4, we could have ((3,2),1,4) in the first stage, then (((3,2),1),4) in the next stage.

2. Allocate progressively using RP from the top of the tree. In the above example, allocate between ((3,2),1) and 4 using RP. Then, allocate between (3,2) and 1 using RP. And finally, allocate between 3 and 2.

## Alternative Methodology

It is possible to do clustering and recursive allocation in one pass. The idea is to start at the root of the tree, with all assets in one portfolio. At every stage we split the portfolio into two by removing the asset that is farthest from all other assets in the distance metric. We do this by computing for each asset its minimum distance to other assets. Then we find the asset that has the maximum of the minimum distances and remove it from the portfolio. We allocate using RP between this asset and the remaining portfolio. This process is repeated till there is only one asset remaining in the portfolio. This essentailly recreates agglomerative clustering in the reverse direction.

Code below follows this method.

## Helper Funcs

In [199]:
function ivp(v::Vector{Float64})
    iv = 1.0 ./ v
    return iv ./ sum(iv)
end

ivp (generic function with 3 methods)

Split a given vector of indices (id) into an index (k) that is farthest from the rest of the indices (id).

In [203]:
function split(D::Matrix{Float64}, id::Union{UnitRange{Int64}, Vector{Int64}})
    if length(id) == 1
        return nothing, id
    end
    tmp = []
    for i in id
        push!(tmp, minimum([D[i,k] for k in setdiff(id, i)]))
    end
    k = id[argmax(tmp)]
    id = setdiff(id, k)
    return id, k
end

split (generic function with 1 method)

Compute HRP weights by recursive RP allocation between splits.

In [211]:
function hrp(Σ, D)
    id = 1:size(D,2)
    w = fill(1.0, size(D,2))
    while length(id) > 1
        id, k = split(D, id)
        v1 = clustervar(Σ, id)
        v2 = Σ[k,k]
        α = ivp([v1,v2])[1]
        w[id] .= w[id] * α
        w[k] = w[k] * (1. - α) 
    end
    return w
end

hrp (generic function with 2 methods)

## A Bigger Example
Generate fake data for a portfolio of 10 assets.

In [212]:
R = rand(LKJ(10,1), 1)[1]
σ = rand(0.1:0.01:0.2, 10)
Σ = diagm(σ)*R*diagm(σ) 
D = sqrt.((1.0 .- R)*0.5)

10×10 Matrix{Float64}:
 0.0       0.55444   0.596279  0.871755  …  0.383599  0.851367  0.484048
 0.55444   0.0       0.526299  0.850921     0.583789  0.629766  0.673496
 0.596279  0.526299  0.0       0.739134     0.551224  0.662068  0.636274
 0.871755  0.850921  0.739134  0.0          0.89054   0.537583  0.768226
 0.764139  0.81803   0.651709  0.610314     0.75588   0.686476  0.759294
 0.682054  0.806884  0.673949  0.819175  …  0.521068  0.766023  0.628537
 0.896789  0.820001  0.71908   0.562226     0.783188  0.442969  0.838506
 0.383599  0.583789  0.551224  0.89054      0.0       0.824285  0.598282
 0.851367  0.629766  0.662068  0.537583     0.824285  0.0       0.73743
 0.484048  0.673496  0.636274  0.768226     0.598282  0.73743   0.0

In [214]:
wrp = ivp(Σ)
whrp = hrp(Σ, D)
[wrp whrp]

10×2 Matrix{Float64}:
 0.0740929  0.0508197
 0.0664989  0.0909951
 0.12248    0.13186
 0.0830661  0.128253
 0.142048   0.183589
 0.12248    0.103041
 0.0740929  0.0607706
 0.106694   0.0731804
 0.0664989  0.0772713
 0.142048   0.10022